In [6]:
%matplotlib inline

from urllib.request import urlretrieve

import matplotlib.pyplot as plt
import nengo
from nengo import spa
import numpy as np
import tensorflow as tf

import nengo_dl

from scipy.signal import decimate

In [11]:
import math
import numpy as np
import cv2
import argparse
from glob import glob

In [17]:
from src.io.psee_loader import PSEELoader

In [21]:
from src.visualize import vis_utils as vis
from src.io.psee_loader import PSEELoader

In [38]:
def play_files_parallel(td_files, labels=None, delta_t=50000, skip=0):
    """
    Plays simultaneously files and their boxes in a rectangular format.
    
    """
    print(td_files)
    # open the video object for the input files
    videos = [PSEELoader(td_file) for td_file in td_files]
    # use the naming pattern to find the corresponding box file
    box_videos = [PSEELoader(glob(td_file.split('_td.dat')[0] +  '*.npy')[0]) for td_file in td_files]
    print('Y')
    
    height, width = videos[0].get_size()
    labelmap = vis.LABELMAP if height == 240 else vis.LABELMAP_LARGE

    # optionally skip n minutes in all videos
    for v in videos + box_videos:
        v.seek_time(skip)

    # preallocate a grid to display the images
    size_x = int(math.ceil(math.sqrt(len(videos))))
    size_y = int(math.ceil(len(videos) / size_x))
    frame = np.zeros((size_y * height, width * size_x, 3), dtype=np.uint8)

    cv2.namedWindow('out', cv2.WINDOW_NORMAL)

    # while all videos have something to read
    while not sum([video.done for video in videos]):

        # load events and boxes from all files
        events = [video.load_delta_t(delta_t) for video in videos]
        box_events = [box_video.load_delta_t(delta_t) for box_video in box_videos]
        for index, (evs, boxes) in enumerate(zip(events, box_events)):
            y, x = divmod(index, size_x)
            # put the visualization at the right spot in the grid
            im = frame[y * height:(y + 1) * height, x * width: (x + 1) * width]
            # call the visualization functions
            im = vis.make_binary_histo(evs, img=im, width=width, height=height)

            vis.draw_bboxes(im, boxes, labelmap=labelmap)

        # display the result
        cv2.imshow('out', frame)
        cv2.waitKey(1)

play_files_parallel(["Prophesee_Dataset_n_cars/n-cars_train/cars/obj_004396_td.dat"], delta_t=10)

['Prophesee_Dataset_n_cars/n-cars_train/cars/obj_004396_td.dat']
HI


IndexError: list index out of range

In [31]:
#play_files_parallel(["data/train/moorea_2019-02-18_000_td_427500000_487500000_td.dat"], delta_t=10)
play_files_parallel(["Prophesee_Dataset_n_cars/n-cars_train/cars/obj_004396_td.dat"], delta_t=10)

IndexError: list index out of range